In [1]:
import os
import dxcam_cpp as dxcam
from src.utils.windowtools import (
    fuzzy_window_search,
    calculate_aspect_ratio,
    check_aspect_ratio_validity,
    get_monitor_number_from_coords,
    normalise_coords_to_monitor
)
from src.utils.helpers import (
    pre_process,
    pre_process_distbox,
)
from src.models import get_model, get_default_model_type, get_model_info
import matplotlib.pyplot as plt
from easyocr import Reader
import numpy as np
import cv2
import tkinter as tk
import threading
import time as systime
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

🔧 Model Configuration: Default model type set to 'optimized'


In [2]:
coords = fuzzy_window_search("asphalt")

monitor_id = get_monitor_number_from_coords(coords)

normalised_coords = normalise_coords_to_monitor(coords, monitor_id)

aspect_ratio = calculate_aspect_ratio(normalised_coords)

check_aspect_ratio_validity(aspect_ratio)
print(coords)

[(0, 0, 2560, 1392)]
1
The aspect ratio is reasonable.
(0, 0, 2560, 1392)


In [3]:
# Global vars
camera = dxcam.create(device_idx=0, output_idx=monitor_id)
capturing = True
time = 0
elapsed_ms = 0
percentage = 0

# Inference time tracking
inference_times = []
total_loops = 0
avg_inference_time = 0

reader = Reader(['en'], gpu=True)  # Still needed for DIST detection

In [4]:
# Grab a frame from the camera
window = camera.grab()

# Extract coordinates from the coords variable
x1, y1, x2, y2 = normalised_coords

capture_coords = (x1, y1, x2, int(y1 + (y2 - y1) / 3.4))

camera.start(region=capture_coords, target_fps=90)

In [5]:
def start_capturing():
    global capturing
    capturing = True

def stop_capturing():
    global capturing
    capturing = False

def toggle_pin():
    global root, is_pinned
    is_pinned = not is_pinned
    if is_pinned:
        root.wm_attributes("-topmost", True)
        pin_button.config(text="📌 Unpin", bg="#ff6b6b")
    else:
        root.wm_attributes("-topmost", False)
        pin_button.config(text="📌 Pin", bg="#4ecdc4")

def update_ui():
    global time, elapsed_ms, percentage, avg_inference_time, inference_times
    
    # Update labels efficiently (only if values changed)
    time_label.config(text=f"Time: {time}")
    elapsed_label.config(text=f"Loop: {elapsed_ms:.1f}ms")
    
    # Smart status display
    if percentage and percentage != "0%":
        percentage_label.config(text=f"Distance: {percentage}", fg="#2ecc71")
        status_label.config(text="🏁 Race in progress", fg="#2ecc71")
    else:
        percentage_label.config(text="Distance: --", fg="#95a5a6")
        status_label.config(text="⏸️ Race not in progress", fg="#95a5a6")
    
    # Performance metrics
    if inference_times:
        current_inference = inference_times[-1] if inference_times else 0
        inference_label.config(text=f"Inference: {current_inference:.1f}ms")
        avg_inference_label.config(text=f"Avg: {avg_inference_time:.1f}ms")
    else:
        inference_label.config(text="Inference: --")
        avg_inference_label.config(text="Avg: --")
    
    # Schedule next update (reduced frequency to avoid performance impact)
    root.after(150, update_ui)

def create_ui():
    global root, time_label, elapsed_label, percentage_label, status_label
    global avg_inference_label, inference_label, pin_button, is_pinned
    
    root = tk.Tk()
    root.title("ALU Timing Tool")
    root.geometry("280x320")
    root.resizable(False, False)
    
    # Set up the window style
    root.configure(bg="#2c3e50")
    is_pinned = False
    
    # Header with pin button
    header_frame = tk.Frame(root, bg="#2c3e50")
    header_frame.pack(fill="x", padx=10, pady=5)
    
    title_label = tk.Label(header_frame, text="ALU Timing Tool", 
                          font=("Helvetica", 12, "bold"), fg="#ecf0f1", bg="#2c3e50")
    title_label.pack(side="left")
    
    pin_button = tk.Button(header_frame, text="📌 Pin", command=toggle_pin, 
                          bg="#4ecdc4", fg="white", font=("Helvetica", 8),
                          relief="flat", padx=8, pady=2)
    pin_button.pack(side="right")
    
    # Control buttons
    button_frame = tk.Frame(root, bg="#2c3e50")
    button_frame.pack(fill="x", padx=10, pady=5)
    
    start_button = tk.Button(button_frame, text="▶️ Start", command=start_capturing, 
                            bg="#27ae60", fg="white", font=("Helvetica", 11, "bold"),
                            relief="flat", padx=15, pady=8)
    start_button.pack(side="left", padx=(0, 5), fill="x", expand=True)

    stop_button = tk.Button(button_frame, text="⏹️ Stop", command=stop_capturing, 
                           bg="#e74c3c", fg="white", font=("Helvetica", 11, "bold"),
                           relief="flat", padx=15, pady=8)
    stop_button.pack(side="right", padx=(5, 0), fill="x", expand=True)
    
    # Status section
    status_frame = tk.Frame(root, bg="#34495e", relief="flat", bd=1)
    status_frame.pack(fill="x", padx=10, pady=5)
    
    status_label = tk.Label(status_frame, text="⏸️ Race not in progress", 
                           font=("Helvetica", 10, "bold"), fg="#95a5a6", bg="#34495e")
    status_label.pack(pady=8)
    
    # Main metrics
    metrics_frame = tk.Frame(root, bg="#2c3e50")
    metrics_frame.pack(fill="x", padx=10, pady=5)
    
    # Time (placeholder for future use)
    time_label = tk.Label(metrics_frame, text=f"Time: {time}", 
                         font=("Helvetica", 10), fg="#ecf0f1", bg="#2c3e50")
    time_label.pack(anchor="w", pady=2)
    
    # Distance percentage
    percentage_label = tk.Label(metrics_frame, text="Distance: --", 
                               font=("Helvetica", 11, "bold"), fg="#95a5a6", bg="#2c3e50")
    percentage_label.pack(anchor="w", pady=2)
    
    # Performance section
    perf_frame = tk.Frame(root, bg="#34495e", relief="flat", bd=1)
    perf_frame.pack(fill="x", padx=10, pady=5)
    
    perf_title = tk.Label(perf_frame, text="Performance", 
                         font=("Helvetica", 9, "bold"), fg="#bdc3c7", bg="#34495e")
    perf_title.pack(anchor="w", padx=8, pady=(5, 0))
    
    # Loop timing
    elapsed_label = tk.Label(perf_frame, text=f"Loop: {elapsed_ms:.1f}ms", 
                            font=("Helvetica", 9), fg="#ecf0f1", bg="#34495e")
    elapsed_label.pack(anchor="w", padx=8, pady=1)
    
    # Inference timing
    inference_label = tk.Label(perf_frame, text="Inference: --", 
                              font=("Helvetica", 9), fg="#ecf0f1", bg="#34495e")
    inference_label.pack(anchor="w", padx=8, pady=1)
    
    avg_inference_label = tk.Label(perf_frame, text="Avg: --", 
                                  font=("Helvetica", 9), fg="#ecf0f1", bg="#34495e")
    avg_inference_label.pack(anchor="w", padx=8, pady=(1, 5))
    
    # Start the UI update loop
    update_ui()
    
    # Make the window appear on top initially
    root.lift()
    root.focus_force()
    
    root.mainloop()

# Initialize UI state variables
is_pinned = False
root = None

ui_thread = threading.Thread(target=create_ui, daemon=True)
ui_thread.start()

In [6]:
# --- Load the Trained Model using centralized system ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

# Try to load the configured model first, with fallback options
model = None
model_name = "unknown"

try:
    # First try the centralized model system
    model = get_model()
    model_name = get_default_model_type()
    
    # Try to load the optimized model weights
    try:
        model.load_state_dict(torch.load('percentage_cnn_optimized.pth', map_location=device))
        print(f"Successfully loaded {model_name} model with optimized weights on {device}.")
    except FileNotFoundError:
        print(f"Optimized weights not found, trying legacy weights...")
        model.load_state_dict(torch.load('percentage_cnn.pth', map_location=device))
        print(f"Successfully loaded {model_name} model with legacy weights on {device}.")
    
    model = model.to(device)
    
except Exception as e:
    print(f"Failed to load centralized model: {e}")
    
    # Fallback to hardcoded model loading
    try:
        # Legacy SimpleCNN fallback
        class SimpleCNN(nn.Module):
            def __init__(self, num_classes=100):
                super(SimpleCNN, self).__init__()
                self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
                self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
                self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
                self.fc1 = nn.Linear(32 * 16 * 16, 512)
                self.fc2 = nn.Linear(512, num_classes)
                self.relu = nn.ReLU()

            def forward(self, x):
                x = self.pool(self.relu(self.conv1(x)))
                x = self.pool(self.relu(self.conv2(x)))
                x = x.view(-1, 32 * 16 * 16)
                x = self.relu(self.fc1(x))
                x = self.fc2(x)
                return x
        
        model = SimpleCNN(num_classes=100).to(device)
        try:
            model.load_state_dict(torch.load('percentage_cnn_optimized.pth', map_location=device))
            model_name = "SimpleCNN (fallback with optimized weights)"
        except FileNotFoundError:
            model.load_state_dict(torch.load('percentage_cnn.pth', map_location=device))
            model_name = "SimpleCNN (fallback with legacy weights)"
        print(f"Successfully loaded fallback model on {device}.")
    except Exception as fallback_error:
        print(f"Fallback model loading also failed: {fallback_error}")
        print("Error: No model could be loaded. Please ensure model files are available.")
        model = None

if model is not None:
    model.eval()  # Set the model to evaluation mode
    print(f"Model architecture: {model.__class__.__name__}")
    print(f"Model device: {next(model.parameters()).device}")
    
    # 🚀 PERFORMANCE OPTIMIZATIONS 🚀
    
    # 1. Enable cudnn benchmarking for consistent convolution algorithms
    if device.type == 'cuda':
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        print("✅ Enabled cuDNN benchmark mode")
    
    # 2. Disable gradient computation globally (already in eval mode, but this is extra)
    torch.set_grad_enabled(False)
    print("✅ Disabled gradient computation")
    
    # 3. Try to compile the model with torch.jit for optimization
    try:
        # Create a dummy input for scripting
        dummy_input = torch.randn(1, 1, 64, 64).to(device)
        model = torch.jit.script(model)
        print("✅ Model compiled with TorchScript")
        
        # Warm up the compiled model
        for _ in range(5):
            with torch.no_grad():
                _ = model(dummy_input)
        print("✅ Model warmed up")
        
    except Exception as jit_error:
        print(f"⚠️ TorchScript compilation failed: {jit_error}")
        print("   Continuing with eager mode...")
    
    # 4. Set memory allocation strategy
    if device.type == 'cuda':
        torch.cuda.empty_cache()
        print("✅ Cleared CUDA cache")

# --- Define Image Transforms ---
# These must be the same as the transforms used during training
data_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

print("🏁 Model loading and optimization complete!")

🔧 Using device: cuda
📋 Using default model type: 'optimized'
Successfully loaded optimized model with optimized weights on cuda.
Model architecture: OptimizedPercentageCNN
Model device: cuda:0
✅ Enabled cuDNN benchmark mode
✅ Disabled gradient computation
✅ Model compiled with TorchScript
✅ Model warmed up
✅ Cleared CUDA cache
🏁 Model loading and optimization complete!
✅ Model warmed up
✅ Cleared CUDA cache
🏁 Model loading and optimization complete!


In [7]:
def the_loop():
    global dist_box, capturing, textarray, camera, percentage, elapsed_ms, total_loops

    # Start the loop
    while capturing:
        if capturing:
            start_time = systime.perf_counter()
            total_loops += 1
            
            # Initialize cnn_result to avoid UnboundLocalError
            cnn_result = None
            
            window = camera.get_latest_frame()
            height, width, _ = window.shape
            top_right_region = window[50:height, 0:int(width * 0.35)]

            if dist_box is None:
                preprocessed_region = pre_process(top_right_region)
                results = reader.readtext(preprocessed_region)
                
                dist_found = False
                dist_bbox = None
                dist_index = -1
                
                # Find DIST
                for i, (bbox, text, confidence) in enumerate(results):
                    if "dist" in text.lower() and not dist_found:
                        dist_bbox = np.array(bbox)
                        dist_index = i
                        dist_found = True
                
                # If we found DIST, look for percentage
                if dist_found:
                    dist_x0, dist_y0 = np.min(dist_bbox[:, 0]), np.min(dist_bbox[:, 1])
                    dist_x1, dist_y1 = np.max(dist_bbox[:, 0]), np.max(dist_bbox[:, 1])
                    dist_center_y = (dist_y0 + dist_y1) / 2
                    
                    best_percentage_match = None
                    best_score = 0
                    
                    # Look for percentage indicators with more flexible criteria
                    for j, (bbox, text, confidence) in enumerate(results):
                        if j == dist_index:  # Skip the DIST box itself
                            continue
                            
                        bbox_array = np.array(bbox)
                        nx0, ny0 = np.min(bbox_array[:, 0]), np.min(bbox_array[:, 1])
                        nx1, ny1 = np.max(bbox_array[:, 0]), np.max(bbox_array[:, 1])
                        bbox_center_y = (ny0 + ny1) / 2
                        
                        # More flexible matching criteria
                        text_clean = text.strip().replace(' ', '').replace(',', '').replace('.', '')
                        
                        # Check if it looks like a percentage
                        has_percent = '%' in text_clean
                        has_numbers = any(char.isdigit() for char in text_clean)
                        ends_with_7 = text_clean.endswith('7')  # Sometimes % is read as 7
                        
                        # Position criteria (more flexible)
                        reasonable_y_distance = abs(bbox_center_y - dist_center_y) < 50
                        to_the_right = nx0 > dist_x0
                        reasonable_x_distance = (nx0 - dist_x1) < 200
                        
                        # Calculate a score for this match
                        score = 0
                        if has_percent:
                            score += 50
                        if has_numbers:
                            score += 20
                        if ends_with_7:
                            score += 10
                        if reasonable_y_distance:
                            score += 30
                        if to_the_right:
                            score += 20
                        if reasonable_x_distance:
                            score += 10
                        
                        # Add confidence boost
                        score += confidence * 10
                        
                        if score > best_score and score > 40:
                            best_score = score
                            best_percentage_match = (j, bbox, text, confidence)
                    
                    # If we found a good percentage match
                    if best_percentage_match is not None:
                        j, next_bbox, next_text, next_confidence = best_percentage_match
                        
                        # Calculate combined bounding box
                        next_box = np.array(next_bbox)
                        nx0, ny0 = np.min(next_box[:, 0]), np.min(next_box[:, 1])
                        nx1, ny1 = np.max(next_box[:, 0]), np.max(next_box[:, 1])
                        
                        # Extend bounding box to include both with some padding
                        x0 = int(min(dist_x0, nx0)) - 5
                        y0 = int(min(dist_y0, ny0)) - 5
                        x1 = int(max(dist_x1, nx1)) + 5
                        y1 = int(max(dist_y1, ny1)) + 5
                        
                        # Ensure bounds are within image
                        x0 = max(0, x0)
                        y0 = max(0, y0)
                        x1 = min(top_right_region.shape[1], x1)
                        y1 = min(top_right_region.shape[0], y1)
                    else:
                        # Fallback: just use DIST box with some expansion
                        x0 = int(dist_x0) - 10
                        y0 = int(dist_y0) - 10
                        x1 = int(dist_x1) + 100
                        y1 = int(dist_y1) + 30
                        
                        # Ensure bounds are within image
                        x0 = max(0, x0)
                        y0 = max(0, y0)
                        x1 = min(top_right_region.shape[1], x1)
                        y1 = min(top_right_region.shape[0], y1)
                    
                    # Create the final bounding box
                    dist_box = np.array([[x0, y0], [x1, y0], [x1, y1], [x0, y1]])
            
            clear_output(wait=True)
            
            # If we have the bounding box, crop the image
            if dist_box is not None:
                roi = top_right_region[int(dist_box[0][1]):int(dist_box[2][1]), int(dist_box[0][0]):int(dist_box[1][0])]
                roi = roi[:, int(roi.shape[1] * 23 / 40):]

                # Preprocess the cropped image for CNN
                preprocessed_region = pre_process_distbox(roi, for_cnn=True)

                # Use CNN for recognition
                cnn_result = predict_with_cnn(preprocessed_region)

            # Process CNN prediction
            if cnn_result is not None:
                predicted_percentage, confidence = cnn_result
                
                if confidence >= CONFIDENCE_THRESHOLD:
                    percentage = f"{predicted_percentage}%"
                else:
                    dist_box = None
                    percentage = ""
            else:
                dist_box = None
                percentage = ""
            
            end_time = systime.perf_counter()
            elapsed_ms = (end_time - start_time) * 1000

            systime.sleep(0.04)

In [8]:
# Run the main loop
the_loop()

NameError: name 'dist_box' is not defined